In [1]:
import matplotlib.pyplot as plt
import numpy as np

import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from grav_lens import get_datasets
from grav_lens.models import test_model_image

In [3]:
from os.path import join
home_data = join("..", "data")
max_files = 1000
batch_size = 16

train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=max_files,
    home=home_data)

Using data folder: ..\data\1


In [6]:
from utils import CustomScaler

# --- Ejemplo de uso del CustomScaler ---
scaler = CustomScaler()

# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)

# Normalizar un batch de datos
for X_batch, y_batch in train_dataset.take(1):
    y_batch_scaled = scaler.transform(y_batch)
    print("Scaled batch:", y_batch_scaled.shape)

# Desescalar para volver a los valores originales
y_batch_original = scaler.inverse_transform(y_batch_scaled)
print("Original batch:", y_batch_original.shape)

train_dataset.map()


Scaled batch: (32, 128, 128, 1)
Original batch: (32, 128, 128, 1)


In [7]:
# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)
# Guardar el scaler ajustado
scaler.save_scaler('scaler.pkl')

# Cargar el scaler cuando lo necesites
scaler.load_scaler('scaler.pkl')

In [11]:
scaler.transform(train_dataset)

AttributeError: '_PrefetchDataset' object has no attribute 'numpy'

In [ ]:
# Función para aplicar el pipeline a cada imagen en el dataset
def process_dataset_pipeline(dataset, threshold_factor=1, n_gaussians=5, threshold_value=2.0):
    processed_results = []
    
    for X_batch, y_batch in dataset:
        # Asegurarse de que y_batch tiene la forma (batch, 128, 128, 1)
        for i in range(y_batch.shape[0]):  # Iterar sobre el batch
            image = y_batch[i, :, :, 0]  # Extraer una imagen 2D (128, 128)
            
            # Ejecutar el pipeline de procesamiento
            result = process_image_pipeline(image, threshold_factor, n_gaussians, threshold_value)
            
            # Almacenar el resultado
            processed_results.append(result)
    
    return processed_results

# Aplicar la pipeline al dataset normalizado
processed_train = process_dataset_pipeline(train_dataset_scaled, threshold_factor=1, n_gaussians=5, threshold_value=2.0)

# Puedes hacer lo mismo para val_dataset_scaled y test_dataset_scaled


In [ ]:
# Visualizar la imagen original y filtrada de un ejemplo
import matplotlib.pyplot as plt

def visualize_results(result, original_image):
    plt.figure(figsize=(12, 5))

    # Imagen original
    plt.subplot(1, 3, 1)
    plt.imshow(original_image, cmap='viridis')
    plt.title("Imagen original")
    plt.colorbar()

    # Imagen filtrada
    plt.subplot(1, 3, 2)
    plt.imshow(result['filtered_image'].numpy(), cmap='viridis')
    plt.title("Imagen filtrada (FFT)")
    plt.colorbar()

    # Imagen con thresholding
    plt.subplot(1, 3, 3)
    plt.imshow(result['thresholded_image'].numpy(), cmap='viridis')
    plt.title("Imagen con threshold")
    plt.colorbar()

    plt.show()

# Ejemplo con un resultado del train dataset
example_result = processed_train[0]
original_image = y_batch[0, :, :, 0].numpy()  # Asegúrate de tener la imagen original
visualize_results(example_result, original_image)
